In [ ]:
import pandas as pd   #importing libraries


In [ ]:
df=pd.read_csv('/content/problem_traveler_raw.csv')    # reading data

In [ ]:
df.head()                           # top 5 rows from dataframe

,PNR,origination_city,destination_city,orig_city_cd,dest_city_cd,origination_lat,origination_lon,destination_lat,destination_lon,fl_grfare_usd_am,fl_valid_id,fl_mile_ct,fl_cost_per_mile_usd_am,carry_a2_cd,fl_no,seg_arr_local_time,seg_dprt_local_time,seg_arr_utc_time,seg_dprt_utc_time
0,921597,PHX,MIA,PHX,MIA,33.435036,-112.000164,25.796000,-80.278234,550.95,1,1981,0.27,AA,772,5/14/2018 16:33,5/14/2018 9:03,5/14/2018 23:33,5/14/2018 14:03
1,921597,MIA,PUJ,MIA,PUJ,25.796000,-80.278234,18.562477,-68.363998,252.80,1,909,0.27,AA,2593,5/14/2018 21:23,5/14/2018 19:00,5/15/2018 2:23,5/14/2018 23:00
2,921597,PUJ,MIA,PUJ,MIA,18.562477,-68.363998,25.796000,-80.278234,252.81,1,909,0.27,AA,1317,5/18/2018 9:41,5/18/2018 7:00,5/18/2018 13:41,5/18/2018 12:00
3,921597,MIA,PHX,MIA,PHX,25.796000,-80.278234,33.435036,-112.000164,550.95,1,1981,0.27,AA,272,5/18/2018 15:59,5/18/2018 13:55,5/18/2018 20:59,5/18/2018 20:55
4,492751,PHX,SAN,PHX,SAN,33.435036,-112.000164,32.731938,-117.197312,249.44,1,302,0.82,AA,465,2/5/2018 8:52,2/5/2018 8:35,2/5/2018 15:52,2/5/2018 16:35


In [ ]:
 #extracting the date and time components from the 'seg_dprt_local_time' and 'seg_arr_local_time
df['dep_Dates'] = pd.to_datetime(df['seg_dprt_local_time']).dt.date
df['dep_Time'] = pd.to_datetime(df['seg_dprt_local_time']).dt.time
df['ar_Dates'] = pd.to_datetime(df['seg_arr_local_time']).dt.date
df['ar_Time'] = pd.to_datetime(df['seg_arr_local_time']).dt.time

In [ ]:
df['fl_grfare_usd_am'].describe()

count      134.000000
mean       938.326418
std       1829.222120
min      -1569.600000
25%         44.992500
50%        330.340000
75%        755.945000
max      11500.010000
Name: fl_grfare_usd_am, dtype: float64

**Filter the dataframe by checking fl_grfare_usd_am is greater than 0, 'fl_valid_id' is equal to 1, and fl_mile_ct is greater than 0. As these can not be negative**

In [ ]:
df = df.loc[(df['fl_grfare_usd_am']>0) & (df['fl_valid_id']==1) & (df['fl_mile_ct']>0)].reset_index(drop=True)

**Aggregate total trip spend and total mile count**

In [ ]:

agg_data = df.groupby("PNR").agg({
    "fl_grfare_usd_am": "sum",
    "fl_mile_ct": "sum",
}).reset_index()


In [ ]:
agg_data

,PNR,fl_grfare_usd_am,fl_mile_ct
0,111318,2740.18,6081
1,252241,1472.43,2131
2,276662,2848.88,8863
3,279194,2112.04,5395
4,295064,10966.53,5279
5,309555,12404.17,18898
6,333260,1773.44,2554
7,335664,10640.98,12991
8,435290,1109.90,3040
9,492751,505.30,604


**Number of valid trips associated with each PNR**


In [ ]:
valid_trips_segment_wise = df.groupby(['PNR'])['fl_valid_id'].sum().reset_index(name='valid_trips_segment')

print(valid_trips_segment_wise)

       PNR  valid_trips_segment
0   111318                    4
1   252241                    1
2   276662                    5
3   279194                    3
4   295064                    1
5   309555                    4
6   333260                    2
7   335664                    4
8   435290                    2
9   492751                    2
10  597788                    2
11  612481                    2
12  756904                    2
13  819915                    4
14  823594                    1
15  875677                    4
16  921597                    4
17  936809                    3
18  961754                    5
19  966615                    2
20  981010                    2
21  993257                    6
22  995277                    6
23  999160                    2
24  999890                    2


**Total valid trips**

In [ ]:
total_valid_trip=valid_trips_segment_wise['valid_trips_segment'].sum()

In [ ]:
total_valid_trip


75

**Count the number of valid trips  (For unique PNR)**

In [ ]:


valid_trip_count = df[df["fl_valid_id"] == 1]["PNR"].nunique()

In [ ]:
valid_trip_count

25

**Calculating no. of distinct cities**

In [ ]:
set_org = set(df["orig_city_cd"])
set_dest = set(df["dest_city_cd"])

In [ ]:
cities=set_org.union(set_dest)

In [ ]:
# Count distinct cities visited

distinct_city_count = len(cities)-1     ### excluding the home city therefore subtracted by 1


In [ ]:
distinct_city_count

29

**distinct city count is 29**

**Infer home city code**

In [ ]:


home_city_code = df["orig_city_cd"].value_counts().idxmax()

In [ ]:
home_city_code

'PHX'

**home city is PHX**

**Qualifying each trip as one way trip or round trip**

In [ ]:
df

,PNR,origination_city,destination_city,orig_city_cd,dest_city_cd,origination_lat,origination_lon,destination_lat,destination_lon,fl_grfare_usd_am,...,seg_arr_local_time,seg_dprt_local_time,seg_arr_utc_time,seg_dprt_utc_time,trip_type,dep_Dates,dep_Time,ar_Dates,ar_Time,travel_history
0,921597,PHX,MIA,PHX,MIA,33.435036,-112.000164,25.796000,-80.278234,550.95,...,5/14/2018 16:33,5/14/2018 9:03,5/14/2018 23:33,5/14/2018 14:03,Round Trip,2018-05-14,09:03:00,2018-05-14,16:33:00,PHX->SFO->SFO->LIH->LIH->LAX->LAX->PHX
1,921597,MIA,PUJ,MIA,PUJ,25.796000,-80.278234,18.562477,-68.363998,252.80,...,5/14/2018 21:23,5/14/2018 19:00,5/15/2018 2:23,5/14/2018 23:00,Round Trip,2018-05-14,19:00:00,2018-05-14,21:23:00,EWR->PHX
2,921597,PUJ,MIA,PUJ,MIA,18.562477,-68.363998,25.796000,-80.278234,252.81,...,5/18/2018 9:41,5/18/2018 7:00,5/18/2018 13:41,5/18/2018 12:00,Round Trip,2018-05-18,07:00:00,2018-05-18,09:41:00,PHX->NYC->NYC->PHX->EWR->EYW->EYW->EWR->EWR->PHX
3,921597,MIA,PHX,MIA,PHX,25.796000,-80.278234,33.435036,-112.000164,550.95,...,5/18/2018 15:59,5/18/2018 13:55,5/18/2018 20:59,5/18/2018 20:55,Round Trip,2018-05-18,13:55:00,2018-05-18,15:59:00,PHX->SEA->SEA->WAS->WAS->PHX
4,492751,PHX,SAN,PHX,SAN,33.435036,-112.000164,32.731938,-117.197312,249.44,...,2/5/2018 8:52,2/5/2018 8:35,2/5/2018 15:52,2/5/2018 16:35,Round Trip,2018-02-05,08:35:00,2018-02-05,08:52:00,LON->PHX
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70,335664,LON,NYC,LHR,JFK,51.469603,-0.453566,40.642335,-73.788170,2809.51,...,3/28/2018 19:15,3/28/2018 16:15,3/28/2018 19:15,3/28/2018 21:15,Round Trip,2018-03-28,16:15:00,2018-03-28,19:15:00,NaN
71,252241,EWR,PHX,EWR,PHX,40.689071,-74.178753,33.435036,-112.000164,1472.43,...,3/29/2018 22:36,3/29/2018 19:55,3/30/2018 3:36,3/30/2018 2:55,Round Trip,2018-03-29,19:55:00,2018-03-29,22:36:00,NaN
72,335664,EWR,PHX,EWR,PHX,40.689071,-74.178753,33.435036,-112.000164,1462.44,...,3/29/2018 22:36,3/29/2018 19:55,3/30/2018 3:36,3/30/2018 2:55,Round Trip,2018-03-29,19:55:00,2018-03-29,22:36:00,NaN
73,435290,PHX,MIA,PHX,MIA,33.435036,-112.000164,25.796000,-80.278234,723.27,...,6/26/2018 16:28,6/26/2018 8:49,6/26/2018 23:28,6/26/2018 13:49,Round Trip,2018-06-26,08:49:00,2018-06-26,16:28:00,NaN


**Sort data by departure and arrival date and time**

In [ ]:
sorted_data = df.sort_values(by=['dep_Dates', 'dep_Time', 'ar_Dates','ar_Time'])

**Create a new column 'travel_history' to store combined origin and destination cities PNR-wise**

In [ ]:

travel_history = sorted_data.groupby('PNR').apply(
    lambda x: ','.join(x['origination_city'] + ',' + x['destination_city'])
).reset_index(name='travel_sequence')

In [ ]:
travel_history


,PNR,travel_sequence
0,111318,"PHX,SFO,SFO,LIH,LIH,LAX,LAX,PHX"
1,252241,"EWR,PHX"
2,276662,"PHX,NYC,NYC,PHX,EWR,EYW,EYW,EWR,EWR,PHX"
3,279194,"PHX,SEA,SEA,WAS,WAS,PHX"
4,295064,"LON,PHX"
5,309555,"PHX,EWR,EWR,DEL,DEL,EWR,EWR,PHX"
6,333260,"MSY,SFO,SFO,PHX"
7,335664,"PHX,LON,PHX,EWR,LON,NYC,EWR,PHX"
8,435290,"PHX,MIA,MIA,BDA"
9,492751,"PHX,SAN,SAN,PHX"


**Add 'trip_type' column based on first and last element of 'travel_sequence'. If travel Sequence start and ends with PHX then it is considered as round trip else one way trip**

In [ ]:

travel_history['trip_type'] = travel_history['travel_sequence'].apply(
    lambda x: 'Round Trip' if x.startswith('PHX') and x.endswith('PHX') else 'One Way'
)


In [ ]:
travel_history

,PNR,travel_sequence,trip_type
0,111318,"PHX,SFO,SFO,LIH,LIH,LAX,LAX,PHX",Round Trip
1,252241,"EWR,PHX",One Way
2,276662,"PHX,NYC,NYC,PHX,EWR,EYW,EYW,EWR,EWR,PHX",Round Trip
3,279194,"PHX,SEA,SEA,WAS,WAS,PHX",Round Trip
4,295064,"LON,PHX",One Way
5,309555,"PHX,EWR,EWR,DEL,DEL,EWR,EWR,PHX",Round Trip
6,333260,"MSY,SFO,SFO,PHX",One Way
7,335664,"PHX,LON,PHX,EWR,LON,NYC,EWR,PHX",Round Trip
8,435290,"PHX,MIA,MIA,BDA",One Way
9,492751,"PHX,SAN,SAN,PHX",Round Trip
